    *****************************************************
    Programmer:    Adeyemi Adedoyin Simeon
    Location:      Offa, Kwara State, Nigeria
    Contact:       adeyemi.sa1@gmail.com, +2348064555381
    Github:        https://github.com/SimeonDee
    Date:          6th April, 2020, 03:16 AM
    Version:       1.3
    *****************************************************

# Python Project to Scrap Saved Contacts .vcf file and extract Names, Phone Numbers and also allows one to Seacrh for saved contacts.
 

This project was inspired by my need to find some numbers when my mobile phone got spoilt. Although i had the contacts exported to an external memory card, i still could not access nor find certain contacts without getting hold of another mobile phone to import the stored contacts to (which at the moment was not even available). 

I obviously would not want to borrow a friend's mobile phone and bombard his/her contacts with my own contacts, especially when all the efforts would only be just finding a few contacts.

So, I wrote this program, hoping that it would be useful to someone else somewhere, facing the same or similar ordeal.

In [26]:
import pandas as pd

class SavedPhonebookExtractor:
    """
    Extracts contact names and phone-numbers from saved phone contact .vcf file and allows you to search for specific contacts using name or phone number
    """
    def __init__(self, saved_contacts_file=None):
        """        
        * This extracts names and phone numbers of contacts saved in .vcf file 
        format and return it as pandas Dataframe.
        * It also supports searching for specific contact.
        * Enables recall of last search results
        """
        self.extracted_contacts_ = None
        self.last_search_results_ = []
        self.contacts_ = None
        
        # Opening the saved contacts .vcf file
        with open(saved_contacts_file) as f:
            lines = [x.strip() for x in f.readlines()]
            txt = ' '.join(lines)
            self.contacts_ = txt.split(' END:VCARD ')
            
        
    def extract_numbers_from_contacts(self):
        """
        Extract phone numbers from a given 'savedContactFile' in .vcf format
        
        Argument: 
        * saved_contacts_file: a string holding the path to the saved .vcf file
        """

        # Actual Extraction of Contact Names and Phone numbers
        contTemp = [x.replace('FN:', '_').replace('TEL;CELL:', '_') for x in self.contacts_]
        contTemp = [x.split(sep='_')[1:3] for x in contTemp]

        for i in range(len(contTemp)):
            if len(contTemp[i]) == 1 and 'TEL' in contTemp[i][0]:
                contTemp[i] = contTemp[i][0].replace('TEL;CELL:', '_', 1).replace('TEL;CELL;PREF:', '_', 1).replace('TEL;OTHER:','_', 1).replace('TEL;HOME:','_', 1)
                contTemp[i] = contTemp[i][0].replace('TEL;WORK:', '_', 1).replace('TEL;X-Mobile:', '_', 1).replace('TEL;X-Other:', '_', 1).replace('TEL;X-Work:', '_', 1)
                contTemp[i] = contTemp[i][0].replace('TEL;X-Home:', '_', 1).replace('TEL;X-Afeez:', '_', 1).replace('TEL;X-Joshua:', '_', 1).replace('TEL;', '_', 1)
                contTemp[i] = contTemp[i].split('_')

            if len(contTemp[i]) == 1:
                contTemp[i] = ['NIL', contTemp[i][0]]

            if 'TEL' in contTemp[i][0]:
                contTemp[i][0] = contTemp[i][0].replace(' TEL;CELL:', '_', 1).replace(' TEL;CELL;PREF:', '_', 1).replace(' TEL;OTHER:','_', 1).replace(' TEL;HOME:','_', 1)
                contTemp[i][0] = contTemp[i][0].replace(' TEL;WORK:', '_', 1).replace(' TEL;X-Mobile:', '_', 1).replace(' TEL;X-Other:', '_', 1).replace(' TEL;X-Work:', '_', 1)
                contTemp[i][0] = contTemp[i][0].replace(' TEL;X-Home:', '_', 1).replace(' TEL;X-Afeez:', '_', 1).replace(' TEL;X-Joshua:', '_', 1)

                t = []
                t.extend(contTemp[i][0].split('_'))
                t.extend(contTemp[i][1:])
                contTemp[i] = t
                t = []

            if len(contTemp[i]) > 2:
                contTemp[i] = [contTemp[i][0], ', '.join(contTemp[i][1:])]

            if 'TEL' in contTemp[i][1]:
                contTemp[i][1] = contTemp[i][1].replace(' TEL;CELL:', ', ').replace(' TEL;CELL;PREF:', ', ').replace(' TEL;OTHER:', ', ').replace(' TEL;HOME:', ', ')
                contTemp[i][1] = contTemp[i][1].replace(' TEL;X-Mobile:', ', ').replace(' TEL;X-Other:', ', ').replace(' TEL;X-Work:', ', ').replace(' TEL;X-Home:', ', ')
                contTemp[i][1] = contTemp[i][1].replace(' TEL;HOME:', ', ').replace(' TEL;WORK:', ', ').replace(' TEL;X-Afeez:', ', ').replace(' TEL;X-Joshua:', ', ').replace(' TEL;', ', ')

            if ' PHOTO;' in contTemp[i][1]:
                contTemp[i][1] = contTemp[i][1].split(' PHOTO;')[0]

            if ';;' in contTemp[i][1]:
                contTemp[i][1] = contTemp[i][1].replace(';;', '')


        # Converting extracted contacts to pd.Dataframe
        dico = {'Names':[x.strip() for [x,_] in contTemp], 'TEL':[y.strip() for [_,y] in contTemp]}
        idx = list(range(1, len(contTemp)+1))
        extracted_contacts = pd.DataFrame(dico, index=idx)

        # Sorting the contacts by name
        extracted_contacts.sort_values(by='Names', inplace=True)
        extracted_contacts.index = list(range(1, len(extracted_contacts) + 1))
        
        self.extracted_contacts_ = extracted_contacts
        print(f'Total Contacts Extracted: {extracted_contacts.shape[0]}')

        return self.extracted_contacts_
    
    
    def find_contact_from_df(self, contacts_df, search_term='', search_term_type='name'):
        """
        Find contacts related to 'search_term' from saved contacts stored as pandas Dataframe
        
        Arguments:
        * contacts: A pandas Dataframe obj containing columns 'Names' and 'TEL'
        
        * search_term (String): 
            A search term to find from contacts whose value can be a 'name' or 'phone' number. 
        
        * search_term_type (String): what to find. 
            Possible Values: {'name', 'phone'}
            The kind of search term supplied.
        
            NOTE: If field='name', then 'search_term' must be name of contact to find
                but if field='phone' then 'search_term' must be set to a Phone number.
        """
        found_flag = False
        if type(contacts_df) == pd.DataFrame and search_term != '':
            self.last_search_results_.clear() # reset
            
            if search_term_type == 'name':    
                for i,val in enumerate(contacts_df.Names):
                    if search_term.upper() in val.upper():
                        found_flag = True
                        self.last_search_results_.append(str(val) + ': ' + contacts_df.iloc[i]['TEL'])
                        print(f"{val} : {contacts_df.iloc[i]['TEL']}")
                        
            elif search_term_type == 'phone':
                for i,val in enumerate(contacts_df.TEL):
                    if search_term.strip() in val.replace(' ', ''):
                        found_flag = True
                        self.last_search_results_.append(contacts_df.iloc[i]['Names'] + ': ' + contacts_df.iloc[i]['TEL'])
                        print(f"{contacts_df.iloc[i]['Names']} : {contacts_df.iloc[i]['TEL']}")
                

            if found_flag == False:
                self.last_search_results_.append('Sorry, ' + search_term + ' is not in the contacts.' )
                print(f'Sorry, {search_term} is not in the contacts.')

            return

        elif type(contacts_df) == pd.DataFrame and search_term == '':
            return contacts_df

        elif type(contacts_df) != pd.DataFrame:
            print("Contacts has to be a pandas dataframe object contating 'Names' and 'TEL' columns.")
            return
        
    
    def find_contact_from_stored_phonebook(self, phonebook_file, search_term='', search_term_type='name'):
        """
        Find phone number of 'search_term' from 'phonebook' stored as vcf
        
        Arguments:
        * phonebook_file: (String), the path to the '.vcf' stored phone-contacts filename
        
        * search_term: (String), A search term to find from contacts
        
        * search_term_type (String): what to find. 
            Possible Values: {'name', 'phone'}
            The kind of search term supplied.
        
            NOTE: If field='name', then 'search_term' must be name of contact to find
                but if field='phone' then 'search_term' must be set to a Phone number.
        """
        self.__init__(phonebook_file)
        contacts = self.extract_numbers_from_contacts()
        self.find_contact_from_df(contacts_df=contacts, search_term=search_term, search_term_type=search_term_type)
    
    
    def find_contact(self, search_term='', search_term_type='name'):
        """
        * search_term: (String), A search term to find from contacts
        
        * search_term_type (String): what to find. 
            Possible Values: {'name', 'phone'}
            The kind of search term supplied.
        
            NOTE: If field='name', then 'search_term' must be name of contact to find
                but if field='phone' then 'search_term' must be set to a Phone number.
        """
        if self.extracted_contacts_ is None:
            _ = self.extract_numbers_from_contacts()
        self.find_contact_from_df(contacts_df=self.extracted_contacts_, search_term=search_term, search_term_type=search_term_type)
        
        
    def get_last_search_results(self):
        """
        Displays the last search results
        """
        print('\n'.join(self.last_search_results_))
    
    def save_extracted_contacts_to_excel_file(self, destination_path='contacts.xlsx'):
        self.extracted_contacts_.to_excel(destination_path)
        print('Contacts saved successfully.')
        

# ============ SAMPLE USAGE ==========================

# Instantiating the Class

In [27]:
contacts_extractor = SavedPhonebookExtractor(saved_contacts_file="MyContactsBkUpFrmPhn1.vcf")

# Extracting the Contacts

In [28]:
extracted_contacts = contacts_extractor.extract_numbers_from_contacts()

Total Contacts Extracted: 1070


In [29]:
extracted_contacts.head()

,Names,TEL
1,*606#,606
2,1Gb Airtel BBC,4401
3,3D Epoxy Floor Training @ Ilorin,08060382009
4,4 me alone NCS Mem No,11053
5,419,08104062962


# Saving Extracted contacts in pandas dataframe format to disk as excel file

In [53]:
contacts_extractor.save_extracted_contacts_to_excel_file(destination_path='ExtractedContacts2.xlsx')

Contacts saved successfully.


# Finding Contacts

In [30]:
contacts_extractor.find_contact(search_term='master')

Dayo Master : 08144406255
Masterline Arts painter : 08034665620, 08050871066


# How to search for contact name, given phone number

In [33]:
contacts_extractor.find_contact(search_term='08144406255', search_term_type='phone')

Dayo Master : 08144406255


# Calling other direct access functions

In [34]:
contacts_extractor.find_contact_from_df(contacts_df=extracted_contacts, search_term='master')

Dayo Master : 08144406255
Masterline Arts painter : 08034665620, 08050871066


In [35]:
# Search name
contacts_extractor.find_contact_from_stored_phonebook(phonebook_file='MyContactsBkUpFrmPhn1.vcf', search_term='master')

Total Contacts Extracted: 1070
Dayo Master : 08144406255
Masterline Arts painter : 08034665620, 08050871066


In [37]:
# Cases where search item annot be found i.e. does not exist in contact
contacts_extractor.find_contact_from_stored_phonebook(phonebook_file='MyContactsBkUpFrmPhn1.vcf', search_term='ball')

Total Contacts Extracted: 1070
Sorry, ball is not in the contacts.


In [41]:
# Search phone number
contacts_extractor.find_contact_from_stored_phonebook(phonebook_file='MyContactsBkUpFrmPhn1.vcf', 
                                                      search_term='08050871066', search_term_type='phone')

Total Contacts Extracted: 1070
Masterline Arts painter : 08034665620, 08050871066


# Viewing last search result

In [42]:
contacts_extractor.get_last_search_results()

Masterline Arts painter: 08034665620, 08050871066


# ================= THE END =============================